In [8]:
import pandas as pd
pd.__version__

'0.24.2'

In [10]:
pépé=0
pépé+1

1

In [24]:
import pyspark

spark = pyspark.sql.SparkSession.builder.appName('lololand').getOrCreate()
spark

In [31]:
spark.stop()

In [32]:
import pyspark
sc = pyspark.SparkContext(appName="rien")

import urllib.request as url
f = url.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")

data_file = "./kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

In [33]:
raw_data.count()

494021

In [35]:
raw_data.take(2)

['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.']

In [38]:
type(raw_data)

pyspark.rdd.RDD

In [40]:
# parse data
csv_data = raw_data.map(lambda x: x.split(","))

# separate into different RDDs
normal_csv_data = csv_data.filter(lambda x: x[41]=="normal.")
attack_csv_data = csv_data.filter(lambda x: x[41]!="normal.")
print(normal_csv_data.count())
print(attack_csv_data.count())

97278
396743


In [54]:
normal_csv_data.map(lambda x: int(x[0]))

PythonRDD[10] at RDD at PythonRDD.scala:53

In [56]:
normal_csv_data.map(lambda x: int(x[0])).reduce(lambda x, y: x + y)

21075991

In [57]:
attack_csv_data.map(lambda x: int(x[0])).reduce(lambda x, y: x + y)

2626792

voir https://github.com/jadianes/spark-py-notebooks/blob/master/nb5-rdd-aggregations/nb5-rdd-aggregations.ipynb

In [61]:
attack_csv_data.map(lambda x: int(x[0])).aggregate(
    (0,0),
    (lambda acc, value: (acc[0] + value, acc[1] + 1)), 
    (lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])))

(2626792, 396743)

In [23]:
import findspark
findspark.init()

import pyspark
import random

sc = pyspark.SparkContext(appName="Pi")
num_samples = 100000000

def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1

count = sc.parallelize(range(0, num_samples)).filter(inside).count()

pi = 4 * count / num_samples
print(pi)

sc.stop()

3.1414384


In [17]:
sc = pyspark.SparkContext(appName="rien")

In [18]:
sc

<SparkContext master=local[*] appName=rien>

In [19]:
sc.stop()

In [41]:
def t0 (*a):
    for x in a:
        print(x)

t0(1,2,3)

1
2
3


In [48]:
def t0 (**a):
    for k,v in a.items():
        print(k,v)
    print (a['xx'])

t0(a=1, b=3, xx='boo')

a 1
b 3
xx boo
boo


In [70]:
l=['a','b','c']
map(lambda x: x+'x', l)

# SVD

In [72]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix

rows = sc.parallelize([
    Vectors.sparse(5, {1: 1.0, 3: 7.0}),
    Vectors.dense(2.0, 0.0, 3.0, 4.0, 5.0),
    Vectors.dense(4.0, 0.0, 0.0, 6.0, 7.0)
])

mat = RowMatrix(rows)

# Compute the top 5 singular values and corresponding singular vectors.
svd = mat.computeSVD(5, computeU=True)
U = svd.U       # The U factor is a RowMatrix.
s = svd.s       # The singular values are stored in a local dense vector.
V = svd.V       # The V factor is a local dense matrix.

In [91]:
x=Vectors.sparse(5, {1: 1.0, 3: 7.0})
x.toArray

<bound method SparseVector.toArray of SparseVector(5, {1: 1.0, 3: 7.0})>

In [87]:
rows.collect()

[SparseVector(5, {1: 1.0, 3: 7.0}),
 DenseVector([2.0, 0.0, 3.0, 4.0, 5.0]),
 DenseVector([4.0, 0.0, 0.0, 6.0, 7.0])]

In [81]:
U.numCols(), U.numRows()

(5, 3)

In [83]:
s

DenseVector([13.0293, 5.3686, 2.533, 0.0, 0.0])

In [85]:
V

DenseMatrix(5, 5, [-0.3128, -0.0298, -0.1221, -0.7185, -0.6084, 0.3117, -0.1713, 0.1526, ..., 0.2383, -0.0209, -0.4626, -0.0745, 0.9735, -0.0345, -0.1391, 0.1618], 0)